In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve


In [67]:
pd.options.display.max_columns
df = pd.read_csv('df_all_rs.csv')
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('index', axis=1)
df

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2004-04-14,20301188,1610612746,1610612760,2003,1610612746,87.0,0.423,0.727,0.214,17.0,37.0,1610612760,118.0,0.542,1.000,0.375,32.0,34.0,0
1,2004-04-14,20301184,1610612759,1610612743,2003,1610612759,93.0,0.424,0.679,0.100,15.0,58.0,1610612743,67.0,0.325,0.611,0.222,11.0,47.0,1
2,2004-04-14,20301181,1610612754,1610612741,2003,1610612754,101.0,0.420,0.794,0.316,24.0,58.0,1610612741,96.0,0.420,0.667,0.357,20.0,41.0,1
3,2004-04-14,20301177,1610612764,1610612740,2003,1610612764,78.0,0.375,0.714,0.211,13.0,39.0,1610612740,94.0,0.451,0.600,0.364,24.0,48.0,0
4,2004-04-14,20301179,1610612752,1610612739,2003,1610612752,90.0,0.481,0.714,0.400,13.0,42.0,1610612739,100.0,0.488,0.900,0.364,22.0,40.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2018-10-17,21800011,1610612758,1610612762,2018,1610612758,117.0,0.516,0.667,0.368,17.0,37.0,1610612762,123.0,0.519,0.737,0.481,21.0,44.0,0
19394,2018-10-17,21800012,1610612746,1610612743,2018,1610612746,98.0,0.398,0.833,0.286,21.0,47.0,1610612743,107.0,0.379,0.786,0.333,20.0,56.0,0
19395,2018-10-17,21800013,1610612756,1610612742,2018,1610612756,121.0,0.543,0.875,0.559,35.0,44.0,1610612742,100.0,0.432,0.700,0.303,28.0,38.0,1
19396,2018-10-16,21800001,1610612738,1610612755,2018,1610612738,105.0,0.433,0.714,0.297,21.0,55.0,1610612755,87.0,0.391,0.609,0.192,18.0,47.0,1


In [68]:
df_teams = pd.read_csv('teams.csv')
df_teams = df_teams[['TEAM_ID', 'ABBREVIATION', 'NICKNAME']]
df_teams

,TEAM_ID,ABBREVIATION,NICKNAME
0,1610612737,ATL,Hawks
1,1610612738,BOS,Celtics
2,1610612740,NOP,Pelicans
3,1610612741,CHI,Bulls
4,1610612742,DAL,Mavericks
5,1610612743,DEN,Nuggets
6,1610612745,HOU,Rockets
7,1610612746,LAC,Clippers
8,1610612747,LAL,Lakers
9,1610612748,MIA,Heat


In [69]:
# unimos los dataframes de los partidos y de los equipos para tener el nombre de los equipos en el dataframe
df_merge = pd.merge(left=df,right=df_teams, left_on='HOME_TEAM_ID', right_on='TEAM_ID')
df_merge = pd.merge(left=df_merge,right=df_teams, left_on='VISITOR_TEAM_ID', right_on='TEAM_ID')
df_merge

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,...,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,TEAM_ID_x,ABBREVIATION_x,NICKNAME_x,TEAM_ID_y,ABBREVIATION_y,NICKNAME_y
0,2004-04-14,20301188,1610612746,1610612760,2003,1610612746,87.0,0.423,0.727,0.214,...,0.375,32.0,34.0,0,1610612746,LAC,Clippers,1610612760,OKC,Thunder
1,2003-10-31,20300025,1610612746,1610612760,2003,1610612746,105.0,0.404,0.833,0.381,...,0.346,27.0,42.0,0,1610612746,LAC,Clippers,1610612760,OKC,Thunder
2,2005-01-12,20400514,1610612746,1610612760,2004,1610612746,103.0,0.494,0.815,0.429,...,0.200,18.0,44.0,1,1610612746,LAC,Clippers,1610612760,OKC,Thunder
3,2004-11-03,20400015,1610612746,1610612760,2004,1610612746,114.0,0.629,0.677,0.556,...,0.409,17.0,37.0,1,1610612746,LAC,Clippers,1610612760,OKC,Thunder
4,2006-04-16,20501201,1610612746,1610612760,2005,1610612746,98.0,0.455,0.781,0.375,...,0.438,29.0,36.0,0,1610612746,LAC,Clippers,1610612760,OKC,Thunder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2014-11-24,21400198,1610612766,1610612746,2014,1610612766,92.0,0.413,0.800,0.100,...,0.469,28.0,43.0,0,1610612766,CHA,Hornets,1610612746,LAC,Clippers
19394,2015-12-30,21500474,1610612766,1610612746,2015,1610612766,117.0,0.432,0.889,0.300,...,0.500,27.0,39.0,0,1610612766,CHA,Hornets,1610612746,LAC,Clippers
19395,2017-02-11,21600808,1610612766,1610612746,2016,1610612766,102.0,0.435,0.750,0.351,...,0.412,26.0,50.0,0,1610612766,CHA,Hornets,1610612746,LAC,Clippers
19396,2017-11-18,21700228,1610612766,1610612746,2017,1610612766,102.0,0.391,0.793,0.423,...,0.276,18.0,44.0,1,1610612766,CHA,Hornets,1610612746,LAC,Clippers


In [71]:
#eliminamos las colunas que hemos metidos de mas al hacer los merges
df_merge = df_merge.drop(['TEAM_ID_x','TEAM_ID_y','TEAM_ID_home'], axis=1)
df_merge

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,ABBREVIATION_x,NICKNAME_x,ABBREVIATION_y,NICKNAME_y
0,2004-04-14,20301188,1610612746,1610612760,2003,87.0,0.423,0.727,0.214,17.0,...,0.542,1.000,0.375,32.0,34.0,0,LAC,Clippers,OKC,Thunder
1,2003-10-31,20300025,1610612746,1610612760,2003,105.0,0.404,0.833,0.381,23.0,...,0.535,0.793,0.346,27.0,42.0,0,LAC,Clippers,OKC,Thunder
2,2005-01-12,20400514,1610612746,1610612760,2004,103.0,0.494,0.815,0.429,20.0,...,0.413,0.778,0.200,18.0,44.0,1,LAC,Clippers,OKC,Thunder
3,2004-11-03,20400015,1610612746,1610612760,2004,114.0,0.629,0.677,0.556,33.0,...,0.370,0.750,0.409,17.0,37.0,1,LAC,Clippers,OKC,Thunder
4,2006-04-16,20501201,1610612746,1610612760,2005,98.0,0.455,0.781,0.375,26.0,...,0.541,0.711,0.438,29.0,36.0,0,LAC,Clippers,OKC,Thunder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2014-11-24,21400198,1610612766,1610612746,2014,92.0,0.413,0.800,0.100,21.0,...,0.511,0.800,0.469,28.0,43.0,0,CHA,Hornets,LAC,Clippers
19394,2015-12-30,21500474,1610612766,1610612746,2015,117.0,0.432,0.889,0.300,20.0,...,0.521,0.805,0.500,27.0,39.0,0,CHA,Hornets,LAC,Clippers
19395,2017-02-11,21600808,1610612766,1610612746,2016,102.0,0.435,0.750,0.351,24.0,...,0.471,0.650,0.412,26.0,50.0,0,CHA,Hornets,LAC,Clippers
19396,2017-11-18,21700228,1610612766,1610612746,2017,102.0,0.391,0.793,0.423,20.0,...,0.398,0.684,0.276,18.0,44.0,1,CHA,Hornets,LAC,Clippers


In [73]:
#ordenamos las columnas a nuestro antojo
df_merge = df_merge[['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'ABBREVIATION_x', 'NICKNAME_x',\
                     'ABBREVIATION_y', 'NICKNAME_y', 'SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home', \
                     'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', \
                     'FG3_PCT_away', 'AST_away' ,'REB_away', 'HOME_TEAM_WINS']]
df_merge

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,ABBREVIATION_x,NICKNAME_x,ABBREVIATION_y,NICKNAME_y,SEASON,PTS_home,...,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,0.214,17.0,37.0,118.0,0.542,1.000,0.375,32.0,34.0,0
1,2003-10-31,20300025,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,105.0,...,0.381,23.0,43.0,124.0,0.535,0.793,0.346,27.0,42.0,0
2,2005-01-12,20400514,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2004,103.0,...,0.429,20.0,36.0,92.0,0.413,0.778,0.200,18.0,44.0,1
3,2004-11-03,20400015,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2004,114.0,...,0.556,33.0,38.0,84.0,0.370,0.750,0.409,17.0,37.0,1
4,2006-04-16,20501201,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2005,98.0,...,0.375,26.0,43.0,114.0,0.541,0.711,0.438,29.0,36.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2014-11-24,21400198,1610612766,1610612746,CHA,Hornets,LAC,Clippers,2014,92.0,...,0.100,21.0,33.0,113.0,0.511,0.800,0.469,28.0,43.0,0
19394,2015-12-30,21500474,1610612766,1610612746,CHA,Hornets,LAC,Clippers,2015,117.0,...,0.300,20.0,42.0,122.0,0.521,0.805,0.500,27.0,39.0,0
19395,2017-02-11,21600808,1610612766,1610612746,CHA,Hornets,LAC,Clippers,2016,102.0,...,0.351,24.0,40.0,107.0,0.471,0.650,0.412,26.0,50.0,0
19396,2017-11-18,21700228,1610612766,1610612746,CHA,Hornets,LAC,Clippers,2017,102.0,...,0.423,20.0,49.0,87.0,0.398,0.684,0.276,18.0,44.0,1


In [74]:
#Renombramos algunas columnas
df_merge = df_merge.rename(columns={'GAME_DATE_EST' : 'GAME_DATE',
                                    'ABBREVIATION_x' : 'HOME_TEAM_ABBREVIATION',
                                    'NICKNAME_x' : 'HOME_TEAM_NICKNAME',
                                    'ABBREVIATION_y' : 'VISITOR_TEAM_ABBREVIATION',
                                    'NICKNAME_y' : 'VISITOR_TEAM_NICKNAME',})
df_merge

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,0.214,17.0,37.0,118.0,0.542,1.000,0.375,32.0,34.0,0
1,2003-10-31,20300025,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,105.0,...,0.381,23.0,43.0,124.0,0.535,0.793,0.346,27.0,42.0,0
2,2005-01-12,20400514,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2004,103.0,...,0.429,20.0,36.0,92.0,0.413,0.778,0.200,18.0,44.0,1
3,2004-11-03,20400015,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2004,114.0,...,0.556,33.0,38.0,84.0,0.370,0.750,0.409,17.0,37.0,1
4,2006-04-16,20501201,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2005,98.0,...,0.375,26.0,43.0,114.0,0.541,0.711,0.438,29.0,36.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2014-11-24,21400198,1610612766,1610612746,CHA,Hornets,LAC,Clippers,2014,92.0,...,0.100,21.0,33.0,113.0,0.511,0.800,0.469,28.0,43.0,0
19394,2015-12-30,21500474,1610612766,1610612746,CHA,Hornets,LAC,Clippers,2015,117.0,...,0.300,20.0,42.0,122.0,0.521,0.805,0.500,27.0,39.0,0
19395,2017-02-11,21600808,1610612766,1610612746,CHA,Hornets,LAC,Clippers,2016,102.0,...,0.351,24.0,40.0,107.0,0.471,0.650,0.412,26.0,50.0,0
19396,2017-11-18,21700228,1610612766,1610612746,CHA,Hornets,LAC,Clippers,2017,102.0,...,0.423,20.0,49.0,87.0,0.398,0.684,0.276,18.0,44.0,1


In [76]:
#ordenamos por game date y game_id
df_merge = df_merge.sort_values(['GAME_DATE', 'GAME_ID'])
df_merge

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
11562,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.350,25.0,39.0,74.0,0.408,0.824,0.250,16.0,44.0,1
12344,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.100,20.0,38.0,82.0,0.361,0.810,0.500,14.0,43.0,1
5505,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.350,32.0,46.0,93.0,0.376,0.733,0.280,17.0,46.0,1
11151,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.313,28.0,40.0,75.0,0.366,0.750,0.417,14.0,39.0,1
16941,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.313,20.0,48.0,85.0,0.368,0.552,0.250,17.0,44.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2019-04-10,21801226,1610612749,1610612760,MIL,Bucks,OKC,Thunder,2018,116.0,...,0.326,27.0,53.0,127.0,0.485,0.615,0.426,40.0,53.0,0
5243,2019-04-10,21801227,1610612759,1610612742,SAS,Spurs,DAL,Mavericks,2018,105.0,...,0.333,22.0,53.0,94.0,0.407,0.750,0.297,27.0,42.0,1
3105,2019-04-10,21801228,1610612743,1610612750,DEN,Nuggets,MIN,Timberwolves,2018,99.0,...,0.303,23.0,53.0,95.0,0.429,0.667,0.406,24.0,41.0,1
1971,2019-04-10,21801229,1610612746,1610612762,LAC,Clippers,UTA,Jazz,2018,143.0,...,0.429,34.0,52.0,137.0,0.443,0.879,0.400,31.0,57.0,1


In [77]:
df_merge = df_merge.reset_index()
df_merge = df_merge.drop('index', axis=1)
df_merge

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.350,25.0,39.0,74.0,0.408,0.824,0.250,16.0,44.0,1
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.100,20.0,38.0,82.0,0.361,0.810,0.500,14.0,43.0,1
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.350,32.0,46.0,93.0,0.376,0.733,0.280,17.0,46.0,1
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.313,28.0,40.0,75.0,0.366,0.750,0.417,14.0,39.0,1
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.313,20.0,48.0,85.0,0.368,0.552,0.250,17.0,44.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2019-04-10,21801226,1610612749,1610612760,MIL,Bucks,OKC,Thunder,2018,116.0,...,0.326,27.0,53.0,127.0,0.485,0.615,0.426,40.0,53.0,0
19394,2019-04-10,21801227,1610612759,1610612742,SAS,Spurs,DAL,Mavericks,2018,105.0,...,0.333,22.0,53.0,94.0,0.407,0.750,0.297,27.0,42.0,1
19395,2019-04-10,21801228,1610612743,1610612750,DEN,Nuggets,MIN,Timberwolves,2018,99.0,...,0.303,23.0,53.0,95.0,0.429,0.667,0.406,24.0,41.0,1
19396,2019-04-10,21801229,1610612746,1610612762,LAC,Clippers,UTA,Jazz,2018,143.0,...,0.429,34.0,52.0,137.0,0.443,0.879,0.400,31.0,57.0,1


In [78]:
df_merge.to_csv('df_merged_renamed_droped_sorted_rs.csv')

In [3]:
pd.options.display.max_columns
df = pd.read_csv('df_merged_renamed_droped_sorted_rs.csv')
df = df.drop('Unnamed: 0', axis=1)
df

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.350,25.0,39.0,74.0,0.408,0.824,0.250,16.0,44.0,1
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.100,20.0,38.0,82.0,0.361,0.810,0.500,14.0,43.0,1
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.350,32.0,46.0,93.0,0.376,0.733,0.280,17.0,46.0,1
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.313,28.0,40.0,75.0,0.366,0.750,0.417,14.0,39.0,1
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.313,20.0,48.0,85.0,0.368,0.552,0.250,17.0,44.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2019-04-10,21801226,1610612749,1610612760,MIL,Bucks,OKC,Thunder,2018,116.0,...,0.326,27.0,53.0,127.0,0.485,0.615,0.426,40.0,53.0,0
19394,2019-04-10,21801227,1610612759,1610612742,SAS,Spurs,DAL,Mavericks,2018,105.0,...,0.333,22.0,53.0,94.0,0.407,0.750,0.297,27.0,42.0,1
19395,2019-04-10,21801228,1610612743,1610612750,DEN,Nuggets,MIN,Timberwolves,2018,99.0,...,0.303,23.0,53.0,95.0,0.429,0.667,0.406,24.0,41.0,1
19396,2019-04-10,21801229,1610612746,1610612762,LAC,Clippers,UTA,Jazz,2018,143.0,...,0.429,34.0,52.0,137.0,0.443,0.879,0.400,31.0,57.0,1


In [4]:
#Añadimos la columna VISITOR_TEAM_WINS que necesitaremos mas adelnate
df['VISITOR_TEAM_WINS'] = np.nan
df

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,25.0,39.0,74.0,0.408,0.824,0.250,16.0,44.0,1,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,20.0,38.0,82.0,0.361,0.810,0.500,14.0,43.0,1,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,32.0,46.0,93.0,0.376,0.733,0.280,17.0,46.0,1,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,28.0,40.0,75.0,0.366,0.750,0.417,14.0,39.0,1,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,20.0,48.0,85.0,0.368,0.552,0.250,17.0,44.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2019-04-10,21801226,1610612749,1610612760,MIL,Bucks,OKC,Thunder,2018,116.0,...,27.0,53.0,127.0,0.485,0.615,0.426,40.0,53.0,0,NaN
19394,2019-04-10,21801227,1610612759,1610612742,SAS,Spurs,DAL,Mavericks,2018,105.0,...,22.0,53.0,94.0,0.407,0.750,0.297,27.0,42.0,1,NaN
19395,2019-04-10,21801228,1610612743,1610612750,DEN,Nuggets,MIN,Timberwolves,2018,99.0,...,23.0,53.0,95.0,0.429,0.667,0.406,24.0,41.0,1,NaN
19396,2019-04-10,21801229,1610612746,1610612762,LAC,Clippers,UTA,Jazz,2018,143.0,...,34.0,52.0,137.0,0.443,0.879,0.400,31.0,57.0,1,NaN


In [5]:
def fill_visitor_team_wins(n):
    if n==1:
        return 0
    else:
        return 1

In [6]:
home_team_wins = list(df['HOME_TEAM_WINS'].values)
visitor_team_wins = list(map(fill_visitor_team_wins, home_team_wins))
df['VISITOR_TEAM_WINS'] = pd.Series(visitor_team_wins)
df

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,25.0,39.0,74.0,0.408,0.824,0.250,16.0,44.0,1,0
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,20.0,38.0,82.0,0.361,0.810,0.500,14.0,43.0,1,0
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,32.0,46.0,93.0,0.376,0.733,0.280,17.0,46.0,1,0
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,28.0,40.0,75.0,0.366,0.750,0.417,14.0,39.0,1,0
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,20.0,48.0,85.0,0.368,0.552,0.250,17.0,44.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2019-04-10,21801226,1610612749,1610612760,MIL,Bucks,OKC,Thunder,2018,116.0,...,27.0,53.0,127.0,0.485,0.615,0.426,40.0,53.0,0,1
19394,2019-04-10,21801227,1610612759,1610612742,SAS,Spurs,DAL,Mavericks,2018,105.0,...,22.0,53.0,94.0,0.407,0.750,0.297,27.0,42.0,1,0
19395,2019-04-10,21801228,1610612743,1610612750,DEN,Nuggets,MIN,Timberwolves,2018,99.0,...,23.0,53.0,95.0,0.429,0.667,0.406,24.0,41.0,1,0
19396,2019-04-10,21801229,1610612746,1610612762,LAC,Clippers,UTA,Jazz,2018,143.0,...,34.0,52.0,137.0,0.443,0.879,0.400,31.0,57.0,1,0


In [186]:
df.to_csv('df_merged_renamed_droped_sorted_2_rs.csv')

In [7]:
#Acontinuacion se quieren agregar una serie de columnas al dataframe que añadiran valor a los datos.
#Mas adelante hablaremos de dichas columnas, por ahora vamos a dividir nuestro dataframe por temporadas,
#mas adelante los uniremos de nuevo para trabajar con uno solo
df_rs_03 = df.query("GAME_DATE >= '2003-10-28' and GAME_DATE <='2004-04-14'")
df_rs_04 = df.query("GAME_DATE >= '2004-11-02' and GAME_DATE <='2005-04-20'")
df_rs_05 = df.query("GAME_DATE >= '2005-11-01' and GAME_DATE <='2006-04-19'")
df_rs_06 = df.query("GAME_DATE >= '2006-10-31' and GAME_DATE <='2007-04-18'")
df_rs_07 = df.query("GAME_DATE >= '2007-10-30' and GAME_DATE <='2008-04-16'")
df_rs_08 = df.query("GAME_DATE >= '2008-10-28' and GAME_DATE <='2009-04-16'")
df_rs_09 = df.query("GAME_DATE >= '2009-10-27' and GAME_DATE <='2010-04-14'")
df_rs_10 = df.query("GAME_DATE >= '2010-10-26' and GAME_DATE <='2011-04-13'")
df_rs_11 = df.query("GAME_DATE >= '2011-12-25' and GAME_DATE <='2012-04-26'")
df_rs_12 = df.query("GAME_DATE >= '2012-10-30' and GAME_DATE <='2013-04-17'")
df_rs_13 = df.query("GAME_DATE >= '2013-10-29' and GAME_DATE <='2014-04-16'")
df_rs_14 = df.query("GAME_DATE >= '2014-10-28' and GAME_DATE <='2015-04-15'")
df_rs_15 = df.query("GAME_DATE >= '2015-10-27' and GAME_DATE <='2016-04-13'")
df_rs_16 = df.query("GAME_DATE >= '2016-10-25' and GAME_DATE <='2017-04-12'")
df_rs_17 = df.query("GAME_DATE >= '2017-10-17' and GAME_DATE <='2018-04-11'")
df_rs_18 = df.query("GAME_DATE >= '2018-10-16' and GAME_DATE <='2019-04-10'")

In [8]:
df_rs_03

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,25.0,39.0,74.0,0.408,0.824,0.250,16.0,44.0,1,0
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,20.0,38.0,82.0,0.361,0.810,0.500,14.0,43.0,1,0
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,32.0,46.0,93.0,0.376,0.733,0.280,17.0,46.0,1,0
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,28.0,40.0,75.0,0.366,0.750,0.417,14.0,39.0,1,0
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,20.0,48.0,85.0,0.368,0.552,0.250,17.0,44.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,20.0,43.0,89.0,0.479,0.609,0.438,18.0,39.0,0,1
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,24.0,50.0,92.0,0.408,0.676,0.389,22.0,40.0,0,1
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,28.0,55.0,105.0,0.398,0.593,0.292,27.0,51.0,0,1
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,17.0,37.0,118.0,0.542,1.000,0.375,32.0,34.0,0,1


In [9]:
#Para no ir de golpe, vamos a ir introduciendo las columnas poco a poco
#las columnas van a ser las siguientes:
# % total de victorias que lleva hasta el momento el equipo home -> PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM
# % total de victorias que lleva hasta el momento el equipo visitor -> PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM
# % victorias ultimos 5 partidos jugados el equipo home -> PERCENT_VIC_LAST5_GAMES_HOME_TEAM
# % victorias ultimos 5 partidos jugados el equipo visitor -> PERCENT_VIC_LAST5_GAMES_HOME_TEAM
# % victorias ultimos 10 partidos jugados el equipo home -> PERCENT_VIC_LAST10_GAMES_HOME_TEAM
# % victorias ultimos 10 partidos jugados el equipo visitor -> PERCENT_VIC_LAST10_GAMES_HOME_TEAM

# media puntos que lleva hasta el momento el equipo home -> AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM
# media puntos que lleva hasta el momento el equipo visitor -> AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM
# media puntos 5 partidos jugados el equipo home -> AVG_POINTS_LAST5_HOME_TEAM
# media puntos 5 partidos jugados el equipo visitor -> AVG_POINTS_LAST5_VISITOR_TEAM
# media puntos 10 partidos jugados el equipo home -> AVG_POINTS_LAST10_HOME_TEAM
# media puntos 10 partidos jugados el equipo visitor -> AVG_POINTS_LAST10_HOME_TEAM

# media %FG que lleva hasta el momento el equipo home -> AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM
# media %FG que lleva hasta el momento el equipo visitor -> AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM
# media %FG ultimos 5 partidos jugados el equipo home -> 
# media %FG ultimos 5 partidos jugados el equipo visitor -> 
# media %FG ultimos 10 partidos jugados el equipo home -> 
# media %FG ultimos 10 partidos jugados el equipo visitor -> 

# media %FG3 que lleva hasta el momento el equipo home -> 
# media %FG3 que lleva hasta el momento el equipo visitor -> 
# media %FG3 ultimos 5 partidos jugados el equipo home -> 
# media %FG3 ultimos 5 partidos jugados el equipo visitor -> 
# media %FG3 ultimos 10 partidos jugados el equipo home -> 
# media %FG3 ultimos 10 partidos jugados el equipo visitor -> 

#Empezaremos por las dos primeras:
#Añadimos la columna VISITOR_TEAM_WINS que necesitaremos mas adelnate
df_rs_03['PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM'] = np.nan
df_rs_03['PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/3233091470.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/3233091470.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,74.0,0.408,0.824,0.250,16.0,44.0,1,0,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,82.0,0.361,0.810,0.500,14.0,43.0,1,0,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,93.0,0.376,0.733,0.280,17.0,46.0,1,0,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,75.0,0.366,0.750,0.417,14.0,39.0,1,0,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,85.0,0.368,0.552,0.250,17.0,44.0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,89.0,0.479,0.609,0.438,18.0,39.0,0,1,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,92.0,0.408,0.676,0.389,22.0,40.0,0,1,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,105.0,0.398,0.593,0.292,27.0,51.0,0,1,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,118.0,0.542,1.000,0.375,32.0,34.0,0,1,NaN,NaN


In [208]:
def test(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe primero para un equipo(home) y despues para el otro(visitor)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    #df_p_v_a_home = df_p_v_a.query("HOME_TEAM_ID == "+str(home_team))#+" or VISITOR_TEAM_ID=="+str(home_team))
    #print(df_p_v_a_home)
    #el numero total de filas es la cantidad de partidos que llevan esta temporada
    total_partidosH = int(len(df_p_v_a_home))
    #print(total_partidosH)
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    #print(df_p_v_a_homeH)
    victorias_as_homeH = int(df_p_v_a_homeH[df_p_v_a_homeH['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #victorias como visitante
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    victorias_as_visitorH = int(df_p_v_a_homeV[df_p_v_a_homeV['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #total victorias
    victorias_totalH = victorias_as_homeH + victorias_as_visitorH
    #El porcentaje de victorias para el equipo que juega como local hasta ese partido es:
    porcentaje_victorias_home_team = (victorias_totalH / total_partidosH) * 100
    #print(porcentaje_victorias_home_team)  
    
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    #el numero total de filas es la cantidad de partidos que llevan esta temporada
    total_partidosV = int(len(df_p_v_a_visitor))
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    victorias_as_homeV = int(df_p_v_a_visitorH[df_p_v_a_visitorH['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #victorias como visitante
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    victorias_as_visitorV = int(df_p_v_a_visitorV[df_p_v_a_visitorV['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #total victorias
    victorias_totalV = victorias_as_homeV + victorias_as_visitorV
    #El porcentaje de victorias para el equipo que juega como visitor hasta ese partido es:
    porcentaje_victorias_visitor_team = (victorias_totalV / total_partidosV) * 100
    #print(porcentaje_victorias_visitor_team)
    

    return [porcentaje_victorias_home_team, porcentaje_victorias_visitor_team]

In [217]:
#Vamos a crear una funcion que dado un game_id me calcule para cada equipo que juega el partido 
#el % de victorias que llevan dicha temporada:
#para eso me basta con saber el numero total de partidos que llevan dicha temporada y el numero de victorias

def porcentaje_victorias_all(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe primero para un equipo(home) y despues para el otro(visitor)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    #df_p_v_a_home = df_p_v_a.query("HOME_TEAM_ID == "+str(home_team))#+" or VISITOR_TEAM_ID=="+str(home_team))
    #print(df_p_v_a_home)
    #el numero total de filas es la cantidad de partidos que llevan esta temporada
    total_partidosH = int(len(df_p_v_a_home))
    #print(total_partidosH)
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    #print(df_p_v_a_homeH)
    victorias_as_homeH = int(df_p_v_a_homeH[df_p_v_a_homeH['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #victorias como visitante
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    victorias_as_visitorH = int(df_p_v_a_homeV[df_p_v_a_homeV['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #total victorias
    victorias_totalH = victorias_as_homeH + victorias_as_visitorH
    #El porcentaje de victorias para el equipo que juega como local hasta ese partido es:
    porcentaje_victorias_home_team = (victorias_totalH / total_partidosH) * 100
    #print(porcentaje_victorias_home_team)  
    
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    #el numero total de filas es la cantidad de partidos que llevan esta temporada
    total_partidosV = int(len(df_p_v_a_visitor))
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    victorias_as_homeV = int(df_p_v_a_visitorH[df_p_v_a_visitorH['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #victorias como visitante
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    victorias_as_visitorV = int(df_p_v_a_visitorV[df_p_v_a_visitorV['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #total victorias
    victorias_totalV = victorias_as_homeV + victorias_as_visitorV
    #El porcentaje de victorias para el equipo que juega como visitor hasta ese partido es:
    porcentaje_victorias_visitor_team = (victorias_totalV / total_partidosV) * 100
    #print(porcentaje_victorias_visitor_team)
    

    return porcentaje_victorias_home_team, porcentaje_victorias_visitor_team


In [18]:
#Vamos a crear dos funciones(una para el equipo que juega como home y otra para el visitor) que dado un game_id 
#el % de victorias que llevan dicha temporada:
#para eso me basta con saber el numero total de partidos que llevan dicha temporada y el numero de victorias

#cada una de estas funciones con un map generara una lista para cada cada columna

def porcentaje_victorias_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    #df_p_v_a_home = df_p_v_a.query("HOME_TEAM_ID == "+str(home_team))#+" or VISITOR_TEAM_ID=="+str(home_team))
    #print(df_p_v_a_home)
    #el numero total de filas es la cantidad de partidos que llevan esta temporada
    total_partidosH = int(len(df_p_v_a_home))
    #print(total_partidosH)
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    #print(df_p_v_a_homeH)
    victorias_as_homeH = int(df_p_v_a_homeH[df_p_v_a_homeH['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #victorias como visitante
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    victorias_as_visitorH = int(df_p_v_a_homeV[df_p_v_a_homeV['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #total victorias
    victorias_totalH = victorias_as_homeH + victorias_as_visitorH
    #El porcentaje de victorias para el equipo que juega como local hasta ese partido es:
    porcentaje_victorias_home_team = (victorias_totalH / total_partidosH) * 100
    #print(porcentaje_victorias_home_team)
    

    return porcentaje_victorias_home_team

def porcentaje_victorias_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    #el numero total de filas es la cantidad de partidos que llevan esta temporada
    total_partidosV = int(len(df_p_v_a_visitor))
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    victorias_as_homeV = int(df_p_v_a_visitorH[df_p_v_a_visitorH['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #victorias como visitante
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    victorias_as_visitorV = int(df_p_v_a_visitorV[df_p_v_a_visitorV['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #total victorias
    victorias_totalV = victorias_as_homeV + victorias_as_visitorV
    #El porcentaje de victorias para el equipo que juega como visitor hasta ese partido es:
    porcentaje_victorias_visitor_team = (victorias_totalV / total_partidosV) * 100
    #print(porcentaje_victorias_visitor_team)
    

    return porcentaje_victorias_visitor_team

In [11]:
#generamos la lista de games ids
game_ids = list(df_rs_03['GAME_ID'].values)
#generamos la lista de la columna PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM
porcentajes_victorias_team_home_rs_03 = list(map(porcentaje_victorias_home_team, game_ids))
#generamos la lista de la columna PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM
porcentajes_victorias_team_visitor_rs_03 = list(map(porcentaje_victorias_visitor_team, game_ids))

In [12]:
#Insertamos los resultados en el dataframe
df_rs_03['PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM'] = pd.Series(porcentajes_victorias_team_home_rs_03)
df_rs_03['PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM'] = pd.Series(porcentajes_victorias_team_visitor_rs_03)

/tmp/ipykernel_4387/2907002619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM'] = pd.Series(porcentajes_victorias_team_home_rs_03)
/tmp/ipykernel_4387/2907002619.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM'] = pd.Series(porcentajes_victorias_team_visitor_rs_03)


In [13]:
#Comprobamos que se han introducido correctamente
df_rs_03

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,74.0,0.408,0.824,0.250,16.0,44.0,1,0,100.000000,0.000000
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,82.0,0.361,0.810,0.500,14.0,43.0,1,0,100.000000,0.000000
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,93.0,0.376,0.733,0.280,17.0,46.0,1,0,100.000000,0.000000
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,75.0,0.366,0.750,0.417,14.0,39.0,1,0,100.000000,0.000000
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,85.0,0.368,0.552,0.250,17.0,44.0,0,1,0.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,89.0,0.479,0.609,0.438,18.0,39.0,0,1,51.219512,35.365854
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,92.0,0.408,0.676,0.389,22.0,40.0,0,1,54.878049,63.414634
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,105.0,0.398,0.593,0.292,27.0,51.0,0,1,50.000000,68.292683
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,118.0,0.542,1.000,0.375,32.0,34.0,0,1,34.146341,45.121951


In [14]:
# Ahora que ya tenemos un modus operandi, vamos a ir ralizando las mismas acciones para el resto de columnas,
# los pasos seran los siguientes:
# 1) Crear pareja de columnas (home y visitor)
# 2) Crear sendas funciones
# 3) Aplicar los map para generar los resultados
# 4) Insertamos los resultado en el dataframe

In [15]:
# 1)
df_rs_03['PERCENT_VIC_LAST5_GAMES_HOME_TEAM'] = np.nan
df_rs_03['PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/3136978825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_LAST5_GAMES_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/3136978825.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.824,0.250,16.0,44.0,1,0,100.000000,0.000000,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.810,0.500,14.0,43.0,1,0,100.000000,0.000000,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.733,0.280,17.0,46.0,1,0,100.000000,0.000000,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.750,0.417,14.0,39.0,1,0,100.000000,0.000000,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.552,0.250,17.0,44.0,0,1,0.000000,100.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,0.609,0.438,18.0,39.0,0,1,51.219512,35.365854,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,0.676,0.389,22.0,40.0,0,1,54.878049,63.414634,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,0.593,0.292,27.0,51.0,0,1,50.000000,68.292683,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,1.000,0.375,32.0,34.0,0,1,34.146341,45.121951,NaN,NaN


In [45]:
# 2)
def porcentaje_victorias_last5_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    #print(df_p_v_a_home)
    
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    df_p_v_a_homeH_last5 = df_p_v_a_homeH.tail(5)
    
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    df_p_v_a_homeV_last5 = df_p_v_a_homeV.tail(5)
    
    df_p_v_a_home = pd.concat([df_p_v_a_homeH_last5, df_p_v_a_homeV_last5])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID']).tail(5)
    total_partidosV = int(len(df_p_v_a_home))
    #print(df_p_v_a_home)
    
    #victorias como local para esos ultimos 5 partidos -> juega en home y gana en home
    victorias_last5_as_homeV_df = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    victorias_last5_as_homeV = int(victorias_last5_as_homeV_df[victorias_last5_as_homeV_df['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #print(victorias_last5_as_homeV)
    #victorias como visitante para esos ultimos 5 partidos --> juega en visitor y gana en visitor
    victorias_last5_as_visitorV_df = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    victorias_last5_as_visitorV = int(victorias_last5_as_visitorV_df[victorias_last5_as_visitorV_df['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #print(victorias_last5_as_visitorV)
    #total victorias
    victorias_last5_totalV = victorias_last5_as_homeV + victorias_last5_as_visitorV
    #print(victorias_last5_totalV)
    #El porcentaje de victorias para el equipo que juega como visitor hasta ese partido es:
    porcentaje_victorias_last5_home_team = (victorias_last5_totalV / total_partidosV) * 100
    #print(porcentaje_victorias_last5_home_team)
    
    return porcentaje_victorias_last5_home_team

def porcentaje_victorias_last5_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    #print(df_p_v_a_visitor)
    
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorH_last5 = df_p_v_a_visitorH.tail(5)
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorV_last5 = df_p_v_a_visitorV.tail(5)
    
    df_p_v_a_visitor = pd.concat([df_p_v_a_visitorH_last5, df_p_v_a_visitorV_last5])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID']).tail(5)
    total_partidosV = int(len(df_p_v_a_visitor))
    #print(df_p_v_a_visitor)
    
    #victorias como local para esos ultimos 5 partidos -> juega en home y gana en home
    victorias_last5_as_homeV_df = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    victorias_last5_as_homeV = int(victorias_last5_as_homeV_df[victorias_last5_as_homeV_df['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #print(victorias_last5_as_homeV)
    #victorias como visitante para esos ultimos 5 partidos --> juega en visitor y gana en visitor
    victorias_last5_as_visitorV_df = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    victorias_last5_as_visitorV = int(victorias_last5_as_visitorV_df[victorias_last5_as_visitorV_df['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #print(victorias_last5_as_visitorV)
    #total victorias
    victorias_last5_totalV = victorias_last5_as_homeV + victorias_last5_as_visitorV
    #print(victorias_last5_totalV)
    #El porcentaje de victorias para el equipo que juega como visitor hasta ese partido es:
    porcentaje_victorias_last5_visitor_team = (victorias_last5_totalV / total_partidosV) * 100
    #print(porcentaje_victorias_last5_visitor_team)
    
    return porcentaje_victorias_last5_visitor_team

In [46]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
porcentajes_victorias_last5_team_home_rs_03 = list(map(porcentaje_victorias_last5_home_team, game_ids))
porcentajes_victorias_last5_team_visitor_rs_03 = list(map(porcentaje_victorias_last5_visitor_team, game_ids))

In [47]:
# 4)
#Insertamos los resultados en el dataframe
df_rs_03['PERCENT_VIC_LAST5_GAMES_HOME_TEAM'] = pd.Series(porcentajes_victorias_last5_team_home_rs_03)
df_rs_03['PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM'] = pd.Series(porcentajes_victorias_last5_team_visitor_rs_03)

/tmp/ipykernel_4387/1898894697.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_LAST5_GAMES_HOME_TEAM'] = pd.Series(porcentajes_victorias_last5_team_home_rs_03)
/tmp/ipykernel_4387/1898894697.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM'] = pd.Series(porcentajes_victorias_last5_team_visitor_rs_03)


In [48]:
df_rs_03

,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.824,0.250,16.0,44.0,1,0,100.000000,0.000000,100.0,0.0
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.810,0.500,14.0,43.0,1,0,100.000000,0.000000,100.0,0.0
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.733,0.280,17.0,46.0,1,0,100.000000,0.000000,100.0,0.0
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.750,0.417,14.0,39.0,1,0,100.000000,0.000000,100.0,0.0
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.552,0.250,17.0,44.0,0,1,0.000000,100.000000,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,0.609,0.438,18.0,39.0,0,1,51.219512,35.365854,40.0,60.0
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,0.676,0.389,22.0,40.0,0,1,54.878049,63.414634,60.0,60.0
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,0.593,0.292,27.0,51.0,0,1,50.000000,68.292683,20.0,60.0
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,1.000,0.375,32.0,34.0,0,1,34.146341,45.121951,20.0,40.0


In [49]:
# 1)
df_rs_03['PERCENT_VIC_LAST10_GAMES_HOME_TEAM'] = np.nan
df_rs_03['PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/170041414.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_LAST10_GAMES_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/170041414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,16.0,44.0,1,0,100.000000,0.000000,100.0,0.0,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,14.0,43.0,1,0,100.000000,0.000000,100.0,0.0,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,17.0,46.0,1,0,100.000000,0.000000,100.0,0.0,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,14.0,39.0,1,0,100.000000,0.000000,100.0,0.0,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,17.0,44.0,0,1,0.000000,100.000000,0.0,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,18.0,39.0,0,1,51.219512,35.365854,40.0,60.0,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,22.0,40.0,0,1,54.878049,63.414634,60.0,60.0,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,27.0,51.0,0,1,50.000000,68.292683,20.0,60.0,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,32.0,34.0,0,1,34.146341,45.121951,20.0,40.0,NaN,NaN


In [50]:
# 2)
def porcentaje_victorias_last10_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    #print(df_p_v_a_home)
    
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    df_p_v_a_homeH_last10 = df_p_v_a_homeH.tail(10)
    
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    df_p_v_a_homeV_last10 = df_p_v_a_homeV.tail(10)
    
    df_p_v_a_home = pd.concat([df_p_v_a_homeH_last10, df_p_v_a_homeV_last10])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID']).tail(10)
    total_partidosV = int(len(df_p_v_a_home))
    #print(df_p_v_a_home)
    
    #victorias como local para esos ultimos 10 partidos -> juega en home y gana en home
    victorias_last10_as_homeV_df = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    victorias_last10_as_homeV = int(victorias_last10_as_homeV_df[victorias_last10_as_homeV_df['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #print(victorias_last10_as_homeV)
    #victorias como visitante para esos ultimos 10 partidos --> juega en visitor y gana en visitor
    victorias_last10_as_visitorV_df = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    victorias_last10_as_visitorV = int(victorias_last10_as_visitorV_df[victorias_last10_as_visitorV_df['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #print(victorias_last10_as_visitorV)
    #total victorias
    victorias_last10_totalV = victorias_last10_as_homeV + victorias_last10_as_visitorV
    #print(victorias_last10_totalV)
    #El porcentaje de victorias para el equipo que juega como visitor hasta ese partido es:
    porcentaje_victorias_last10_home_team = (victorias_last10_totalV / total_partidosV) * 100
    #print(porcentaje_victorias_last10_home_team)
    
    return porcentaje_victorias_last10_home_team

def porcentaje_victorias_last10_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    #print(df_p_v_a_visitor)
    
    #para saber el numero de victorias lo voy a dividir en dos, primero cuando juega como local y 
    #despues le sumare las que consigue tambn como visitante
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorH_last10 = df_p_v_a_visitorH.tail(10)
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorV_last10 = df_p_v_a_visitorV.tail(10)
    
    df_p_v_a_visitor = pd.concat([df_p_v_a_visitorH_last10, df_p_v_a_visitorV_last10])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID']).tail(10)
    total_partidosV = int(len(df_p_v_a_visitor))
    #print(df_p_v_a_visitor)
    
    #victorias como local para esos ultimos 10 partidos -> juega en home y gana en home
    victorias_last10_as_homeV_df = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    victorias_last10_as_homeV = int(victorias_last10_as_homeV_df[victorias_last10_as_homeV_df['HOME_TEAM_WINS'] >= 1]['HOME_TEAM_WINS'].count())
    #print(victorias_last10_as_homeV)
    #victorias como visitante para esos ultimos 10 partidos --> juega en visitor y gana en visitor
    victorias_last10_as_visitorV_df = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    victorias_last10_as_visitorV = int(victorias_last10_as_visitorV_df[victorias_last10_as_visitorV_df['VISITOR_TEAM_WINS'] >= 1]['VISITOR_TEAM_WINS'].count())
    #print(victorias_last10_as_visitorV)
    #total victorias
    victorias_last10_totalV = victorias_last10_as_homeV + victorias_last10_as_visitorV
    #print(victorias_last10_totalV)
    #El porcentaje de victorias para el equipo que juega como visitor hasta ese partido es:
    porcentaje_victorias_last10_visitor_team = (victorias_last10_totalV / total_partidosV) * 100
    #print(porcentaje_victorias_last10_visitor_team)
    
    return porcentaje_victorias_last10_visitor_team

In [51]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
porcentajes_victorias_last10_team_home_rs_03 = list(map(porcentaje_victorias_last10_home_team, game_ids))
porcentajes_victorias_last10_team_visitor_rs_03 = list(map(porcentaje_victorias_last10_visitor_team, game_ids))

In [52]:
# 4)
df_rs_03['PERCENT_VIC_LAST10_GAMES_HOME_TEAM'] = pd.Series(porcentajes_victorias_last10_team_home_rs_03)
df_rs_03['PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM'] = pd.Series(porcentajes_victorias_last10_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/3285968373.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_LAST10_GAMES_HOME_TEAM'] = pd.Series(porcentajes_victorias_last10_team_home_rs_03)
/tmp/ipykernel_4387/3285968373.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM'] = pd.Series(porcentajes_victorias_last10_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,16.0,44.0,1,0,100.000000,0.000000,100.0,0.0,100.0,0.0
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,14.0,43.0,1,0,100.000000,0.000000,100.0,0.0,100.0,0.0
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,17.0,46.0,1,0,100.000000,0.000000,100.0,0.0,100.0,0.0
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,14.0,39.0,1,0,100.000000,0.000000,100.0,0.0,100.0,0.0
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,17.0,44.0,0,1,0.000000,100.000000,0.0,100.0,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,18.0,39.0,0,1,51.219512,35.365854,40.0,60.0,50.0,50.0
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,22.0,40.0,0,1,54.878049,63.414634,60.0,60.0,30.0,80.0
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,27.0,51.0,0,1,50.000000,68.292683,20.0,60.0,40.0,70.0
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,32.0,34.0,0,1,34.146341,45.121951,20.0,40.0,10.0,30.0


In [53]:
#1)
df_rs_03['AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM'] = np.nan
df_rs_03['AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/139143963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/139143963.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,HOME_TEAM_WINS,VISITOR_TEAM_WINS,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,1,0,100.000000,0.000000,100.0,0.0,100.0,0.0,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,1,0,100.000000,0.000000,100.0,0.0,100.0,0.0,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,1,0,100.000000,0.000000,100.0,0.0,100.0,0.0,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,1,0,100.000000,0.000000,100.0,0.0,100.0,0.0,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0,1,0.000000,100.000000,0.0,100.0,0.0,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,0,1,51.219512,35.365854,40.0,60.0,50.0,50.0,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,0,1,54.878049,63.414634,60.0,60.0,30.0,80.0,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,0,1,50.000000,68.292683,20.0,60.0,40.0,70.0,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,0,1,34.146341,45.121951,20.0,40.0,10.0,30.0,NaN,NaN


In [136]:
#2)
def avg_points_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    avg_points_as_homeH = df_p_v_a_homeH['PTS_home']
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    avg_points_as_homeV = df_p_v_a_homeV['PTS_away']
    
    df_p_v_a_home = pd.concat([avg_points_as_homeH, avg_points_as_homeV])
    avg_total_home_team = np.mean(df_p_v_a_home)
    
    return avg_total_home_team

def avg_points_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    avg_points_as_visitorH = df_p_v_a_visitorH['PTS_home']
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    avg_points_as_visitorV = df_p_v_a_visitorV['PTS_away']
    
    df_p_v_a_visitor = pd.concat([avg_points_as_visitorH, avg_points_as_visitorV])
    avg_total_visitor_team = np.mean(df_p_v_a_visitor)
    
    return avg_total_visitor_team


In [137]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_points_team_home_rs_03 = list(map(avg_points_home_team, game_ids))
avg_points_team_visitor_rs_03 = list(map(avg_points_visitor_team, game_ids))


In [139]:
# 4)
df_rs_03['AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM'] = pd.Series(avg_points_team_home_rs_03)
df_rs_03['AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM'] = pd.Series(avg_points_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/110725541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM'] = pd.Series(avg_points_team_home_rs_03)
/tmp/ipykernel_4387/110725541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM'] = pd.Series(avg_points_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,89.000000,74.000000,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,83.000000,82.000000,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,109.000000,93.000000,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,98.000000,74.500000,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.000000,100.000000,0.0,100.0,0.0,100.0,83.000000,85.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,51.219512,35.365854,40.0,60.0,50.0,50.0,88.670732,94.182927,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,54.878049,63.414634,60.0,60.0,30.0,80.0,89.780488,105.195122,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,50.000000,68.292683,20.0,60.0,40.0,70.0,90.731707,98.195122,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,34.146341,45.121951,20.0,40.0,10.0,30.0,94.768293,97.121951,NaN,NaN


In [87]:
#1)
df_rs_03['AVG_POINTS_LAST5_HOME_TEAM'] = np.nan
df_rs_03['AVG_POINTS_LAST5_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/2431206692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_LAST5_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/2431206692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_LAST5_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,89.000000,74.000000,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,83.000000,82.000000,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,109.000000,93.000000,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,98.000000,74.500000,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.000000,100.000000,0.0,100.0,0.0,100.0,83.000000,85.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,51.219512,35.365854,40.0,60.0,50.0,50.0,88.670732,94.182927,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,54.878049,63.414634,60.0,60.0,30.0,80.0,89.780488,105.195122,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,50.000000,68.292683,20.0,60.0,40.0,70.0,90.731707,98.195122,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,34.146341,45.121951,20.0,40.0,10.0,30.0,94.768293,97.121951,NaN,NaN


In [140]:
#2)
def avg_points_last5_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    df_p_v_a_homeH_last5 = df_p_v_a_homeH.tail(5)
    
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    df_p_v_a_homeV_last5 = df_p_v_a_homeV.tail(5)
    
    df_p_v_a_home = pd.concat([df_p_v_a_homeH_last5, df_p_v_a_homeV_last5])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID']).tail(5)
    total_partidosV = int(len(df_p_v_a_home))
    
    #print(df_p_v_a_home)
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    #print(df_p_v_a_homeH)
    avg_points_as_homeH = df_p_v_a_homeH['PTS_home']
    #print(avg_points_as_homeH)
    #if np.isnan(avg_points_as_homeH):
    #    avg_points_as_homeH = 0
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    #print(df_p_v_a_homeV)
    avg_points_as_homeV = df_p_v_a_homeV['PTS_away']
    #print(avg_points_as_homeV)
    
    df_p_v_a_home = pd.concat([avg_points_as_homeH, avg_points_as_homeV])
    avg_total_home_team = np.mean(df_p_v_a_home)    

    return avg_total_home_team

def avg_points_last5_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorH_last5 = df_p_v_a_visitorH.tail(5)
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorV_last5 = df_p_v_a_visitorV.tail(5)
    
    df_p_v_a_visitor = pd.concat([df_p_v_a_visitorH_last5, df_p_v_a_visitorV_last5])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID']).tail(5)
    total_partidosV = int(len(df_p_v_a_visitor))
    
    #print(df_p_v_a_visitor)
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorH)
    avg_points_as_visitorH = df_p_v_a_visitorH['PTS_home']
        
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorV)
    avg_points_as_visitorV = df_p_v_a_visitorV['PTS_away']
    
    df_p_v_a_visitor = pd.concat([avg_points_as_visitorH, avg_points_as_visitorV])
    avg_total_visitor_team = np.mean(df_p_v_a_visitor)

    return avg_total_visitor_team

In [141]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_points_last5_team_home_rs_03 = list(map(avg_points_last5_home_team, game_ids))
avg_points_last5_team_visitor_rs_03 = list(map(avg_points_last5_visitor_team, game_ids))

In [142]:
# 4)
df_rs_03['AVG_POINTS_LAST5_HOME_TEAM'] = pd.Series(avg_points_last5_team_home_rs_03)
df_rs_03['AVG_POINTS_LAST5_VISITOR_TEAM'] = pd.Series(avg_points_last5_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/1905527209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_LAST5_HOME_TEAM'] = pd.Series(avg_points_last5_team_home_rs_03)
/tmp/ipykernel_4387/1905527209.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_LAST5_VISITOR_TEAM'] = pd.Series(avg_points_last5_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PERCENT_VIC_UNITL_THIS_GAME_HOME_TEAM,PERCENT_VIC_UNITL_THIS_GAME_VISITOR_TEAM,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,89.000000,74.000000,89.0,74.0
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,83.000000,82.000000,83.0,82.0
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,109.000000,93.000000,109.0,93.0
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,100.000000,0.000000,100.0,0.0,100.0,0.0,98.000000,74.500000,98.0,74.5
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.000000,100.000000,0.0,100.0,0.0,100.0,83.000000,85.000000,83.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,51.219512,35.365854,40.0,60.0,50.0,50.0,88.670732,94.182927,88.4,94.2
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,54.878049,63.414634,60.0,60.0,30.0,80.0,89.780488,105.195122,95.4,105.0
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,50.000000,68.292683,20.0,60.0,40.0,70.0,90.731707,98.195122,86.2,96.4
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,34.146341,45.121951,20.0,40.0,10.0,30.0,94.768293,97.121951,87.0,105.4


In [143]:
#1)
df_rs_03['AVG_POINTS_LAST10_HOME_TEAM'] = np.nan
df_rs_03['AVG_POINTS_LAST10_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/982310410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_LAST10_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/982310410.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_LAST10_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,100.0,0.0,100.0,0.0,89.000000,74.000000,89.0,74.0,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,100.0,0.0,100.0,0.0,83.000000,82.000000,83.0,82.0,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,100.0,0.0,100.0,0.0,109.000000,93.000000,109.0,93.0,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,100.0,0.0,100.0,0.0,98.000000,74.500000,98.0,74.5,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.0,100.0,0.0,100.0,83.000000,85.000000,83.0,85.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,40.0,60.0,50.0,50.0,88.670732,94.182927,88.4,94.2,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,60.0,60.0,30.0,80.0,89.780488,105.195122,95.4,105.0,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,20.0,60.0,40.0,70.0,90.731707,98.195122,86.2,96.4,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,20.0,40.0,10.0,30.0,94.768293,97.121951,87.0,105.4,NaN,NaN


In [144]:
#2)
def avg_points_last10_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    df_p_v_a_homeH_last10 = df_p_v_a_homeH.tail(10)
    
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    df_p_v_a_homeV_last10 = df_p_v_a_homeV.tail(10)
    
    df_p_v_a_home = pd.concat([df_p_v_a_homeH_last10, df_p_v_a_homeV_last10])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID']).tail(10)
    total_partidosV = int(len(df_p_v_a_home))
    
    #print(df_p_v_a_home)
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    #print(df_p_v_a_homeH)
    avg_points_as_homeH = df_p_v_a_homeH['PTS_home']
    #print(avg_points_as_homeH)
    #if np.isnan(avg_points_as_homeH):
    #    avg_points_as_homeH = 0
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    #print(df_p_v_a_homeV)
    avg_points_as_homeV = df_p_v_a_homeV['PTS_away']
    #print(avg_points_as_homeV)
    
    df_p_v_a_home = pd.concat([avg_points_as_homeH, avg_points_as_homeV])
    avg_total_home_team = np.mean(df_p_v_a_home)    

    return avg_total_home_team

def avg_points_last10_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorH_last10 = df_p_v_a_visitorH.tail(10)
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorV_last10 = df_p_v_a_visitorV.tail(10)
    
    df_p_v_a_visitor = pd.concat([df_p_v_a_visitorH_last10, df_p_v_a_visitorV_last10])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID']).tail(10)
    total_partidosV = int(len(df_p_v_a_visitor))
    
    #print(df_p_v_a_visitor)
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorH)
    avg_points_as_visitorH = df_p_v_a_visitorH['PTS_home']
        
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorV)
    avg_points_as_visitorV = df_p_v_a_visitorV['PTS_away']
    
    df_p_v_a_visitor = pd.concat([avg_points_as_visitorH, avg_points_as_visitorV])
    avg_total_visitor_team = np.mean(df_p_v_a_visitor)

    return avg_total_visitor_team

In [145]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_points_last10_team_home_rs_03 = list(map(avg_points_last10_home_team, game_ids))
avg_points_last10_team_visitor_rs_03 = list(map(avg_points_last10_visitor_team, game_ids))

In [146]:
# 4)
df_rs_03['AVG_POINTS_LAST10_HOME_TEAM'] = pd.Series(avg_points_last10_team_home_rs_03)
df_rs_03['AVG_POINTS_LAST10_VISITOR_TEAM'] = pd.Series(avg_points_last10_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/3988302489.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_LAST10_HOME_TEAM'] = pd.Series(avg_points_last10_team_home_rs_03)
/tmp/ipykernel_4387/3988302489.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_POINTS_LAST10_VISITOR_TEAM'] = pd.Series(avg_points_last10_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PERCENT_VIC_LAST5_GAMES_HOME_TEAM,PERCENT_VIC_LAST5_GAMES_VISITOR_TEAM,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,100.0,0.0,100.0,0.0,89.000000,74.000000,89.0,74.0,89.0,74.0
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,100.0,0.0,100.0,0.0,83.000000,82.000000,83.0,82.0,83.0,82.0
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,100.0,0.0,100.0,0.0,109.000000,93.000000,109.0,93.0,109.0,93.0
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,100.0,0.0,100.0,0.0,98.000000,74.500000,98.0,74.5,98.0,74.5
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.0,100.0,0.0,100.0,83.000000,85.000000,83.0,85.0,83.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,40.0,60.0,50.0,50.0,88.670732,94.182927,88.4,94.2,87.8,94.9
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,60.0,60.0,30.0,80.0,89.780488,105.195122,95.4,105.0,93.4,113.8
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,20.0,60.0,40.0,70.0,90.731707,98.195122,86.2,96.4,88.7,95.9
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,20.0,40.0,10.0,30.0,94.768293,97.121951,87.0,105.4,89.7,101.9


In [148]:
#1)
df_rs_03['AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM'] = np.nan
df_rs_03['AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/2162132115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/2162132115.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,100.0,0.0,89.000000,74.000000,89.0,74.0,89.0,74.0,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,100.0,0.0,83.000000,82.000000,83.0,82.0,83.0,82.0,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,100.0,0.0,109.000000,93.000000,109.0,93.0,109.0,93.0,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,100.0,0.0,98.000000,74.500000,98.0,74.5,98.0,74.5,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.0,100.0,83.000000,85.000000,83.0,85.0,83.0,85.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,50.0,50.0,88.670732,94.182927,88.4,94.2,87.8,94.9,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,30.0,80.0,89.780488,105.195122,95.4,105.0,93.4,113.8,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,40.0,70.0,90.731707,98.195122,86.2,96.4,88.7,95.9,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,10.0,30.0,94.768293,97.121951,87.0,105.4,89.7,101.9,NaN,NaN


In [149]:
#2)
def avg_fg_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    avg_fg_as_homeH = df_p_v_a_homeH['FG_PCT_home']
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    avg_fg_as_homeV = df_p_v_a_homeV['FG_PCT_away']
    
    df_p_v_a_home = pd.concat([avg_fg_as_homeH, avg_fg_as_homeV])
    avg_fg_home_team = np.mean(df_p_v_a_home)
    
    return avg_fg_home_team

def avg_fg_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    avg_fg_as_visitorH = df_p_v_a_visitorH['FG_PCT_home']
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    avg_fg_as_visitorV = df_p_v_a_visitorV['FG_PCT_away']
    
    df_p_v_a_visitor = pd.concat([avg_fg_as_visitorH, avg_fg_as_visitorV])
    avg_fg_visitor_team = np.mean(df_p_v_a_visitor)
    
    return avg_fg_visitor_team


In [150]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_fg_team_home_rs_03 = list(map(avg_fg_home_team, game_ids))
avg_fg_team_visitor_rs_03 = list(map(avg_fg_visitor_team, game_ids))

In [154]:
#4)
df_rs_03['AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM'] = pd.Series(avg_fg_team_home_rs_03)
df_rs_03['AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM'] = pd.Series(avg_fg_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/2663484728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM'] = pd.Series(avg_fg_team_home_rs_03)
/tmp/ipykernel_4387/2663484728.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM'] = pd.Series(avg_fg_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,PERCENT_VIC_LAST10_GAMES_HOME_TEAM,PERCENT_VIC_LAST10_GAMES_VISITOR_TEAM,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,100.0,0.0,89.000000,74.000000,89.0,74.0,89.0,74.0,0.440000,0.408000
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,100.0,0.0,83.000000,82.000000,83.0,82.0,83.0,82.0,0.425000,0.361000
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,100.0,0.0,109.000000,93.000000,109.0,93.0,109.0,93.0,0.506000,0.376000
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,100.0,0.0,98.000000,74.500000,98.0,74.5,98.0,74.5,0.507000,0.387000
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.0,100.0,83.000000,85.000000,83.0,85.0,83.0,85.0,0.400000,0.368000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,50.0,50.0,88.670732,94.182927,88.4,94.2,87.8,94.9,0.437024,0.444463
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,30.0,80.0,89.780488,105.195122,95.4,105.0,93.4,113.8,0.442341,0.460732
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,40.0,70.0,90.731707,98.195122,86.2,96.4,88.7,95.9,0.448683,0.454000
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,10.0,30.0,94.768293,97.121951,87.0,105.4,89.7,101.9,0.428890,0.445549


In [155]:
#1)
df_rs_03['AVG_FGPERCENT_LAST5_HOME_TEAM'] = np.nan
df_rs_03['AVG_FGPERCENT_LAST5_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/2454006404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_LAST5_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/2454006404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_LAST5_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,89.000000,74.000000,89.0,74.0,89.0,74.0,0.440000,0.408000,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,83.000000,82.000000,83.0,82.0,83.0,82.0,0.425000,0.361000,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,109.000000,93.000000,109.0,93.0,109.0,93.0,0.506000,0.376000,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,98.000000,74.500000,98.0,74.5,98.0,74.5,0.507000,0.387000,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,83.000000,85.000000,83.0,85.0,83.0,85.0,0.400000,0.368000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,88.670732,94.182927,88.4,94.2,87.8,94.9,0.437024,0.444463,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,89.780488,105.195122,95.4,105.0,93.4,113.8,0.442341,0.460732,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,90.731707,98.195122,86.2,96.4,88.7,95.9,0.448683,0.454000,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,94.768293,97.121951,87.0,105.4,89.7,101.9,0.428890,0.445549,NaN,NaN


In [160]:
#2)
def avg_fg_last5_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    df_p_v_a_homeH_last5 = df_p_v_a_homeH.tail(5)
    
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    df_p_v_a_homeV_last5 = df_p_v_a_homeV.tail(5)
    
    df_p_v_a_home = pd.concat([df_p_v_a_homeH_last5, df_p_v_a_homeV_last5])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID']).tail(5)
    total_partidosV = int(len(df_p_v_a_home))
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    avg_fg5_as_homeH = df_p_v_a_homeH['FG_PCT_home']
    
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    avg_fg5_as_homeV = df_p_v_a_homeV['FG_PCT_away']
    
    df_p_v_a_home = pd.concat([avg_fg5_as_homeH, avg_fg5_as_homeV])
    avg_fg5_home_team = np.mean(df_p_v_a_home)    

    return avg_fg5_home_team

def avg_fg_last5_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorH_last5 = df_p_v_a_visitorH.tail(5)
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorV_last5 = df_p_v_a_visitorV.tail(5)
    
    df_p_v_a_visitor = pd.concat([df_p_v_a_visitorH_last5, df_p_v_a_visitorV_last5])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID']).tail(5)
    total_partidosV = int(len(df_p_v_a_visitor))
    
    #print(df_p_v_a_visitor)
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorH)
    avg_fg5_as_visitorH = df_p_v_a_visitorH['FG_PCT_home']
        
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorV)
    avg_fg5_as_visitorV = df_p_v_a_visitorV['FG_PCT_away']
    
    df_p_v_a_visitor = pd.concat([avg_fg5_as_visitorH, avg_fg5_as_visitorV])
    avg_fg5_visitor_team = np.mean(df_p_v_a_visitor)

    return avg_fg5_visitor_team

In [161]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_fg5_team_home_rs_03 = list(map(avg_fg_last5_home_team, game_ids))
avg_fg5_team_visitor_rs_03 = list(map(avg_fg_last5_visitor_team, game_ids))

In [162]:
print(avg_fg5_team_home_rs_03)
print(len(avg_fg5_team_home_rs_03) == len(df_rs_03))

[0.44, 0.425, 0.506, 0.507, 0.4, 0.324, 0.425, 0.392, 0.447, 0.317, 0.292, 0.575, 0.391, 0.506, 0.408, 0.389, 0.487, 0.387, 0.40549999999999997, 0.37699999999999995, 0.42199999999999993, 0.4315, 0.37, 0.35850000000000004, 0.4109999999999999, 0.51, 0.44999999999999996, 0.38933333333333336, 0.422, 0.41733333333333333, 0.412, 0.4706666666666666, 0.443, 0.4255, 0.37399999999999994, 0.3786666666666667, 0.3956666666666666, 0.4615, 0.4925, 0.4626666666666667, 0.35450000000000004, 0.42325, 0.38666666666666666, 0.31575000000000003, 0.45875, 0.48233333333333334, 0.4383333333333333, 0.41174999999999995, 0.4065, 0.45299999999999996, 0.36920000000000003, 0.4436, 0.44425, 0.37560000000000004, 0.41274999999999995, 0.41425, 0.43825000000000003, 0.43900000000000006, 0.47700000000000004, 0.45075000000000004, 0.44425000000000003, 0.40679999999999994, 0.37075, 0.3656, 0.4348000000000001, 0.4606, 0.42779999999999996, 0.41839999999999994, 0.3796, 0.4718, 0.421, 0.38300000000000006, 0.3902, 0.4086, 0.4472000

In [163]:
#4)
df_rs_03['AVG_FGPERCENT_LAST5_HOME_TEAM'] = pd.Series(avg_fg5_team_home_rs_03)
df_rs_03['AVG_FGPERCENT_LAST5_VISITOR_TEAM'] = pd.Series(avg_fg5_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/27058921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_LAST5_HOME_TEAM'] = pd.Series(avg_fg5_team_home_rs_03)
/tmp/ipykernel_4387/27058921.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_LAST5_VISITOR_TEAM'] = pd.Series(avg_fg5_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_POINTS_UNTIL_THIS_GAME_HOME_TEAM,AVG_POINTS_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,89.000000,74.000000,89.0,74.0,89.0,74.0,0.440000,0.408000,0.4400,0.4080
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,83.000000,82.000000,83.0,82.0,83.0,82.0,0.425000,0.361000,0.4250,0.3610
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,109.000000,93.000000,109.0,93.0,109.0,93.0,0.506000,0.376000,0.5060,0.3760
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,98.000000,74.500000,98.0,74.5,98.0,74.5,0.507000,0.387000,0.5070,0.3870
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,83.000000,85.000000,83.0,85.0,83.0,85.0,0.400000,0.368000,0.4000,0.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,88.670732,94.182927,88.4,94.2,87.8,94.9,0.437024,0.444463,0.4114,0.4424
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,89.780488,105.195122,95.4,105.0,93.4,113.8,0.442341,0.460732,0.4638,0.4686
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,90.731707,98.195122,86.2,96.4,88.7,95.9,0.448683,0.454000,0.4330,0.4402
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,94.768293,97.121951,87.0,105.4,89.7,101.9,0.428890,0.445549,0.4160,0.4930


In [164]:
#1)
df_rs_03['AVG_FGPERCENT_LAST10_HOME_TEAM'] = np.nan
df_rs_03['AVG_FGPERCENT_LAST10_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/3644774063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_LAST10_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/3644774063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_LAST10_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM,AVG_FGPERCENT_LAST10_HOME_TEAM,AVG_FGPERCENT_LAST10_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,89.0,74.0,89.0,74.0,0.440000,0.408000,0.4400,0.4080,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,83.0,82.0,83.0,82.0,0.425000,0.361000,0.4250,0.3610,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,109.0,93.0,109.0,93.0,0.506000,0.376000,0.5060,0.3760,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,98.0,74.5,98.0,74.5,0.507000,0.387000,0.5070,0.3870,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,83.0,85.0,83.0,85.0,0.400000,0.368000,0.4000,0.3680,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,88.4,94.2,87.8,94.9,0.437024,0.444463,0.4114,0.4424,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,95.4,105.0,93.4,113.8,0.442341,0.460732,0.4638,0.4686,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,86.2,96.4,88.7,95.9,0.448683,0.454000,0.4330,0.4402,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,87.0,105.4,89.7,101.9,0.428890,0.445549,0.4160,0.4930,NaN,NaN


In [165]:
#2)
def avg_fg_last10_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    df_p_v_a_homeH_last10 = df_p_v_a_homeH.tail(10)
    
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    df_p_v_a_homeV_last10 = df_p_v_a_homeV.tail(10)
    
    df_p_v_a_home = pd.concat([df_p_v_a_homeH_last10, df_p_v_a_homeV_last10])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID']).tail(10)
    total_partidosV = int(len(df_p_v_a_home))
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    avg_fg10_as_homeH = df_p_v_a_homeH['FG_PCT_home']
    
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    avg_fg10_as_homeV = df_p_v_a_homeV['FG_PCT_away']
    
    df_p_v_a_home = pd.concat([avg_fg10_as_homeH, avg_fg10_as_homeV])
    avg_fg10_home_team = np.mean(df_p_v_a_home)    

    return avg_fg10_home_team

def avg_fg_last10_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorH_last10 = df_p_v_a_visitorH.tail(10)
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorV_last10 = df_p_v_a_visitorV.tail(10)
    
    df_p_v_a_visitor = pd.concat([df_p_v_a_visitorH_last10, df_p_v_a_visitorV_last10])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID']).tail(10)
    total_partidosV = int(len(df_p_v_a_visitor))
    
    #print(df_p_v_a_visitor)
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorH)
    avg_fg10_as_visitorH = df_p_v_a_visitorH['FG_PCT_home']
        
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorV)
    avg_fg10_as_visitorV = df_p_v_a_visitorV['FG_PCT_away']
    
    df_p_v_a_visitor = pd.concat([avg_fg10_as_visitorH, avg_fg10_as_visitorV])
    avg_fg10_visitor_team = np.mean(df_p_v_a_visitor)

    return avg_fg10_visitor_team

In [166]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_fg10_team_home_rs_03 = list(map(avg_fg_last10_home_team, game_ids))
avg_fg10_team_visitor_rs_03 = list(map(avg_fg_last10_visitor_team, game_ids))

In [167]:
print(avg_fg10_team_home_rs_03)
print(len(avg_fg10_team_home_rs_03) == len(df_rs_03))

[0.44, 0.425, 0.506, 0.507, 0.4, 0.324, 0.425, 0.392, 0.447, 0.317, 0.292, 0.575, 0.391, 0.506, 0.408, 0.389, 0.487, 0.387, 0.40549999999999997, 0.37699999999999995, 0.42199999999999993, 0.4315, 0.37, 0.35850000000000004, 0.4109999999999999, 0.51, 0.44999999999999996, 0.38933333333333336, 0.422, 0.41733333333333333, 0.412, 0.4706666666666666, 0.443, 0.4255, 0.37399999999999994, 0.3786666666666667, 0.3956666666666666, 0.4615, 0.4925, 0.4626666666666667, 0.35450000000000004, 0.42325, 0.38666666666666666, 0.31575000000000003, 0.45875, 0.48233333333333334, 0.4383333333333333, 0.41174999999999995, 0.4065, 0.45299999999999996, 0.36920000000000003, 0.4436, 0.44425, 0.37560000000000004, 0.41274999999999995, 0.41425, 0.43825000000000003, 0.43900000000000006, 0.47700000000000004, 0.45075000000000004, 0.44425000000000003, 0.40679999999999994, 0.37075, 0.3755, 0.4348000000000001, 0.4606, 0.42779999999999996, 0.4246666666666667, 0.37766666666666665, 0.44716666666666666, 0.421, 0.37200000000000005, 

In [168]:
#4)
df_rs_03['AVG_FGPERCENT_LAST10_HOME_TEAM'] = pd.Series(avg_fg10_team_home_rs_03)
df_rs_03['AVG_FGPERCENT_LAST10_VISITOR_TEAM'] = pd.Series(avg_fg10_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/526997355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_LAST10_HOME_TEAM'] = pd.Series(avg_fg10_team_home_rs_03)
/tmp/ipykernel_4387/526997355.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FGPERCENT_LAST10_VISITOR_TEAM'] = pd.Series(avg_fg10_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_POINTS_LAST5_HOME_TEAM,AVG_POINTS_LAST5_VISITOR_TEAM,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM,AVG_FGPERCENT_LAST10_HOME_TEAM,AVG_FGPERCENT_LAST10_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,89.0,74.0,89.0,74.0,0.440000,0.408000,0.4400,0.4080,0.4400,0.4080
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,83.0,82.0,83.0,82.0,0.425000,0.361000,0.4250,0.3610,0.4250,0.3610
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,109.0,93.0,109.0,93.0,0.506000,0.376000,0.5060,0.3760,0.5060,0.3760
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,98.0,74.5,98.0,74.5,0.507000,0.387000,0.5070,0.3870,0.5070,0.3870
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,83.0,85.0,83.0,85.0,0.400000,0.368000,0.4000,0.3680,0.4000,0.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,88.4,94.2,87.8,94.9,0.437024,0.444463,0.4114,0.4424,0.4363,0.4537
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,95.4,105.0,93.4,113.8,0.442341,0.460732,0.4638,0.4686,0.4612,0.5147
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,86.2,96.4,88.7,95.9,0.448683,0.454000,0.4330,0.4402,0.4659,0.4456
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,87.0,105.4,89.7,101.9,0.428890,0.445549,0.4160,0.4930,0.4170,0.4799


In [170]:
#1)
df_rs_03['AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM'] = np.nan
df_rs_03['AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/841350338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/841350338.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM,AVG_FGPERCENT_LAST10_HOME_TEAM,AVG_FGPERCENT_LAST10_VISITOR_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,89.0,74.0,0.440000,0.408000,0.4400,0.4080,0.4400,0.4080,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,83.0,82.0,0.425000,0.361000,0.4250,0.3610,0.4250,0.3610,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,109.0,93.0,0.506000,0.376000,0.5060,0.3760,0.5060,0.3760,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,98.0,74.5,0.507000,0.387000,0.5070,0.3870,0.5070,0.3870,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,83.0,85.0,0.400000,0.368000,0.4000,0.3680,0.4000,0.3680,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,87.8,94.9,0.437024,0.444463,0.4114,0.4424,0.4363,0.4537,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,93.4,113.8,0.442341,0.460732,0.4638,0.4686,0.4612,0.5147,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,88.7,95.9,0.448683,0.454000,0.4330,0.4402,0.4659,0.4456,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,89.7,101.9,0.428890,0.445549,0.4160,0.4930,0.4170,0.4799,NaN,NaN


In [171]:
#2)
def avg_fg3_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    avg_fg3_as_homeH = df_p_v_a_homeH['FG3_PCT_home']
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    avg_fg3_as_homeV = df_p_v_a_homeV['FG3_PCT_away']
    
    df_p_v_a_home = pd.concat([avg_fg3_as_homeH, avg_fg3_as_homeV])
    avg_fg3_home_team = np.mean(df_p_v_a_home)
    
    return avg_fg3_home_team

def avg_fg3_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    avg_fg3_as_visitorH = df_p_v_a_visitorH['FG3_PCT_home']
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    avg_fg3_as_visitorV = df_p_v_a_visitorV['FG3_PCT_away']
    
    df_p_v_a_visitor = pd.concat([avg_fg3_as_visitorH, avg_fg3_as_visitorV])
    avg_fg3_visitor_team = np.mean(df_p_v_a_visitor)
    
    return avg_fg3_visitor_team


In [172]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_fg3_team_home_rs_03 = list(map(avg_fg3_home_team, game_ids))
avg_fg3_team_visitor_rs_03 = list(map(avg_fg3_visitor_team, game_ids))

In [174]:
#4)
df_rs_03['AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM'] = pd.Series(avg_fg3_team_home_rs_03)
df_rs_03['AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM'] = pd.Series(avg_fg3_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/2270045719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM'] = pd.Series(avg_fg3_team_home_rs_03)
/tmp/ipykernel_4387/2270045719.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM'] = pd.Series(avg_fg3_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_POINTS_LAST10_HOME_TEAM,AVG_POINTS_LAST10_VISITOR_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM,AVG_FGPERCENT_LAST10_HOME_TEAM,AVG_FGPERCENT_LAST10_VISITOR_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,89.0,74.0,0.440000,0.408000,0.4400,0.4080,0.4400,0.4080,0.350000,0.250000
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,83.0,82.0,0.425000,0.361000,0.4250,0.3610,0.4250,0.3610,0.100000,0.500000
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,109.0,93.0,0.506000,0.376000,0.5060,0.3760,0.5060,0.3760,0.350000,0.280000
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,98.0,74.5,0.507000,0.387000,0.5070,0.3870,0.5070,0.3870,0.313000,0.333500
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,83.0,85.0,0.400000,0.368000,0.4000,0.3680,0.4000,0.3680,0.313000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,87.8,94.9,0.437024,0.444463,0.4114,0.4424,0.4363,0.4537,0.318878,0.340049
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,93.4,113.8,0.442341,0.460732,0.4638,0.4686,0.4612,0.5147,0.365305,0.347183
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,88.7,95.9,0.448683,0.454000,0.4330,0.4402,0.4659,0.4456,0.343756,0.321244
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,89.7,101.9,0.428890,0.445549,0.4160,0.4930,0.4170,0.4799,0.317146,0.373902


In [175]:
#1)
df_rs_03['AVG_FG3PCT_LAST5_HOME_TEAM'] = np.nan
df_rs_03['AVG_FG3PCT_LAST5_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/3770958418.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_LAST5_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/3770958418.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_LAST5_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM,AVG_FGPERCENT_LAST10_HOME_TEAM,AVG_FGPERCENT_LAST10_VISITOR_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FG3PCT_LAST5_HOME_TEAM,AVG_FG3PCT_LAST5_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.440000,0.408000,0.4400,0.4080,0.4400,0.4080,0.350000,0.250000,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.425000,0.361000,0.4250,0.3610,0.4250,0.3610,0.100000,0.500000,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.506000,0.376000,0.5060,0.3760,0.5060,0.3760,0.350000,0.280000,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.507000,0.387000,0.5070,0.3870,0.5070,0.3870,0.313000,0.333500,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.400000,0.368000,0.4000,0.3680,0.4000,0.3680,0.313000,0.250000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,0.437024,0.444463,0.4114,0.4424,0.4363,0.4537,0.318878,0.340049,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,0.442341,0.460732,0.4638,0.4686,0.4612,0.5147,0.365305,0.347183,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,0.448683,0.454000,0.4330,0.4402,0.4659,0.4456,0.343756,0.321244,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,0.428890,0.445549,0.4160,0.4930,0.4170,0.4799,0.317146,0.373902,NaN,NaN


In [176]:
#2)
def avg_fg3_last5_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    df_p_v_a_homeH_last5 = df_p_v_a_homeH.tail(5)
    
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    df_p_v_a_homeV_last5 = df_p_v_a_homeV.tail(5)
    
    df_p_v_a_home = pd.concat([df_p_v_a_homeH_last5, df_p_v_a_homeV_last5])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID']).tail(5)
    total_partidosV = int(len(df_p_v_a_home))
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    avg_fg35_as_homeH = df_p_v_a_homeH['FG3_PCT_home']
    
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    avg_fg35_as_homeV = df_p_v_a_homeV['FG3_PCT_away']
    
    df_p_v_a_home = pd.concat([avg_fg35_as_homeH, avg_fg35_as_homeV])
    avg_fg35_home_team = np.mean(df_p_v_a_home)    

    return avg_fg35_home_team

def avg_fg3_last5_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorH_last5 = df_p_v_a_visitorH.tail(5)
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorV_last5 = df_p_v_a_visitorV.tail(5)
    
    df_p_v_a_visitor = pd.concat([df_p_v_a_visitorH_last5, df_p_v_a_visitorV_last5])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID']).tail(5)
    total_partidosV = int(len(df_p_v_a_visitor))
    
    #print(df_p_v_a_visitor)
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorH)
    avg_fg35_as_visitorH = df_p_v_a_visitorH['FG3_PCT_home']
        
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorV)
    avg_fg35_as_visitorV = df_p_v_a_visitorV['FG3_PCT_away']
    
    df_p_v_a_visitor = pd.concat([avg_fg35_as_visitorH, avg_fg35_as_visitorV])
    avg_fg35_visitor_team = np.mean(df_p_v_a_visitor)

    return avg_fg35_visitor_team

In [177]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_fg35_team_home_rs_03 = list(map(avg_fg3_last5_home_team, game_ids))
avg_fg35_team_visitor_rs_03 = list(map(avg_fg3_last5_visitor_team, game_ids))

In [178]:
print(avg_fg35_team_home_rs_03)
print(len(avg_fg35_team_home_rs_03) == len(df_rs_03))

[0.35, 0.1, 0.35, 0.313, 0.313, 0.16, 0.1669999999999999, 0.3329999999999999, 0.1669999999999999, 0.231, 0.4, 0.556, 0.238, 0.222, 0.31, 0.28300000000000003, 0.5710000000000001, 0.35, 0.2499999999999999, 0.25933333333333336, 0.375, 0.2485, 0.231, 0.3655, 0.3905, 0.3835, 0.258, 0.32766666666666666, 0.39066666666666666, 0.3426666666666667, 0.25566666666666665, 0.29633333333333334, 0.27133333333333337, 0.35200000000000004, 0.19566666666666666, 0.36933333333333335, 0.39033333333333337, 0.36649999999999994, 0.3115, 0.28933333333333333, 0.30825, 0.37625, 0.25933333333333336, 0.3525, 0.322, 0.37966666666666665, 0.3353333333333333, 0.259, 0.31475, 0.34075, 0.2508, 0.2722, 0.30175, 0.3266, 0.18974999999999997, 0.39075, 0.29974999999999996, 0.32799999999999996, 0.30974999999999997, 0.27525, 0.3765, 0.3946, 0.32974999999999993, 0.3242, 0.3128, 0.244, 0.24059999999999998, 0.2738, 0.33540000000000003, 0.409, 0.41259999999999997, 0.40680000000000005, 0.3496, 0.354, 0.3126, 0.359, 0.3598, 0.2808, 0.4

In [179]:
#4)
df_rs_03['AVG_FG3PCT_LAST5_HOME_TEAM'] = pd.Series(avg_fg35_team_home_rs_03)
df_rs_03['AVG_FG3PCT_LAST5_VISITOR_TEAM'] = pd.Series(avg_fg35_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/398926785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_LAST5_HOME_TEAM'] = pd.Series(avg_fg35_team_home_rs_03)
/tmp/ipykernel_4387/398926785.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_LAST5_VISITOR_TEAM'] = pd.Series(avg_fg35_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_FGPERCENT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FGPERCENT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM,AVG_FGPERCENT_LAST10_HOME_TEAM,AVG_FGPERCENT_LAST10_VISITOR_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FG3PCT_LAST5_HOME_TEAM,AVG_FG3PCT_LAST5_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.440000,0.408000,0.4400,0.4080,0.4400,0.4080,0.350000,0.250000,0.3500,0.2500
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.425000,0.361000,0.4250,0.3610,0.4250,0.3610,0.100000,0.500000,0.1000,0.5000
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.506000,0.376000,0.5060,0.3760,0.5060,0.3760,0.350000,0.280000,0.3500,0.2800
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.507000,0.387000,0.5070,0.3870,0.5070,0.3870,0.313000,0.333500,0.3130,0.3335
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.400000,0.368000,0.4000,0.3680,0.4000,0.3680,0.313000,0.250000,0.3130,0.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,0.437024,0.444463,0.4114,0.4424,0.4363,0.4537,0.318878,0.340049,0.3044,0.4792
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,0.442341,0.460732,0.4638,0.4686,0.4612,0.5147,0.365305,0.347183,0.4136,0.3966
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,0.448683,0.454000,0.4330,0.4402,0.4659,0.4456,0.343756,0.321244,0.3812,0.2542
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,0.428890,0.445549,0.4160,0.4930,0.4170,0.4799,0.317146,0.373902,0.1910,0.3962


In [180]:
#1)
df_rs_03['AVG_FG3PCT_LAST10_HOME_TEAM'] = np.nan
df_rs_03['AVG_FG3PCT_LAST10_VISITOR_TEAM'] = np.nan
df_rs_03

/tmp/ipykernel_4387/123558974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_LAST10_HOME_TEAM'] = np.nan
/tmp/ipykernel_4387/123558974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_LAST10_VISITOR_TEAM'] = np.nan


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM,AVG_FGPERCENT_LAST10_HOME_TEAM,AVG_FGPERCENT_LAST10_VISITOR_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FG3PCT_LAST5_HOME_TEAM,AVG_FG3PCT_LAST5_VISITOR_TEAM,AVG_FG3PCT_LAST10_HOME_TEAM,AVG_FG3PCT_LAST10_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.4400,0.4080,0.4400,0.4080,0.350000,0.250000,0.3500,0.2500,NaN,NaN
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.4250,0.3610,0.4250,0.3610,0.100000,0.500000,0.1000,0.5000,NaN,NaN
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.5060,0.3760,0.5060,0.3760,0.350000,0.280000,0.3500,0.2800,NaN,NaN
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.5070,0.3870,0.5070,0.3870,0.313000,0.333500,0.3130,0.3335,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.4000,0.3680,0.4000,0.3680,0.313000,0.250000,0.3130,0.2500,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,0.4114,0.4424,0.4363,0.4537,0.318878,0.340049,0.3044,0.4792,NaN,NaN
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,0.4638,0.4686,0.4612,0.5147,0.365305,0.347183,0.4136,0.3966,NaN,NaN
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,0.4330,0.4402,0.4659,0.4456,0.343756,0.321244,0.3812,0.2542,NaN,NaN
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,0.4160,0.4930,0.4170,0.4799,0.317146,0.373902,0.1910,0.3962,NaN,NaN


In [181]:
#2)
def avg_fg3_last10_home_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para un equipo(home)
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el home_team
    df1h = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == home_team)]
    df2h = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == home_team)]
    df_p_v_a_home = pd.concat([df1h, df2h])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    df_p_v_a_homeH_last10 = df_p_v_a_homeH.tail(10)
    
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    df_p_v_a_homeV_last10 = df_p_v_a_homeV.tail(10)
    
    df_p_v_a_home = pd.concat([df_p_v_a_homeH_last10, df_p_v_a_homeV_last10])
    df_p_v_a_home = df_p_v_a_home.sort_values(['GAME_DATE', 'GAME_ID']).tail(10)
    total_partidosV = int(len(df_p_v_a_home))
    
    
    df_p_v_a_homeH = df_p_v_a_home.query("HOME_TEAM_ID == " + str(home_team))
    avg_fg310_as_homeH = df_p_v_a_homeH['FG3_PCT_home']
    
        
    df_p_v_a_homeV = df_p_v_a_home.query("VISITOR_TEAM_ID == " + str(home_team))
    avg_fg310_as_homeV = df_p_v_a_homeV['FG3_PCT_away']
    
    df_p_v_a_home = pd.concat([avg_fg310_as_homeH, avg_fg310_as_homeV])
    avg_fg310_home_team = np.mean(df_p_v_a_home)    

    return avg_fg310_home_team

def avg_fg3_last10_visitor_team(game_id):
    #lo primero que hacemos es quedarnos con los id de los equipos de ese game_id
    home_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['HOME_TEAM_ID'])
    #print(home_team)
    visitor_team = int(df_rs_03.query("GAME_ID == " + str(game_id))['VISITOR_TEAM_ID'])
    #print(visitor_team)
    #Acontinuacion filtramos el dataframe para quedarnos solo con los partidos anteriores
    #df_p_v_a = df_rs_03.query("GAME_ID <= " + str(game_id))
    df_p_v_a = df_rs_03[(df_rs_03['GAME_ID'] <= game_id)]
    #print(df_p_v_a)
    #ahora tengo que filtrar este dataframe para el visitor
    #filtro tanto si jugaron en casa como fuera (cada uno de ellos), ya que para un partido puedes juegar como home o visitor
    
    #Vamos con el visitor_team
    df1v = df_p_v_a[(df_p_v_a['HOME_TEAM_ID'] == visitor_team)]
    df2v = df_p_v_a[(df_p_v_a['VISITOR_TEAM_ID'] == visitor_team)]
    df_p_v_a_visitor = pd.concat([df1v, df2v])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID'])
    
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorH_last10 = df_p_v_a_visitorH.tail(10)
    
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    df_p_v_a_visitorV_last10 = df_p_v_a_visitorV.tail(10)
    
    df_p_v_a_visitor = pd.concat([df_p_v_a_visitorH_last10, df_p_v_a_visitorV_last10])
    df_p_v_a_visitor = df_p_v_a_visitor.sort_values(['GAME_DATE', 'GAME_ID']).tail(10)
    total_partidosV = int(len(df_p_v_a_visitor))
    
    #print(df_p_v_a_visitor)
    
    df_p_v_a_visitorH = df_p_v_a_visitor.query("HOME_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorH)
    avg_fg310_as_visitorH = df_p_v_a_visitorH['FG3_PCT_home']
        
    df_p_v_a_visitorV = df_p_v_a_visitor.query("VISITOR_TEAM_ID == " + str(visitor_team))
    #print(df_p_v_a_visitorV)
    avg_fg310_as_visitorV = df_p_v_a_visitorV['FG3_PCT_away']
    
    df_p_v_a_visitor = pd.concat([avg_fg310_as_visitorH, avg_fg310_as_visitorV])
    avg_fg310_visitor_team = np.mean(df_p_v_a_visitor)

    return avg_fg310_visitor_team

In [182]:
# 3)
game_ids = list(df_rs_03['GAME_ID'].values)
avg_fg310_team_home_rs_03 = list(map(avg_fg3_last10_home_team, game_ids))
avg_fg310_team_visitor_rs_03 = list(map(avg_fg3_last10_visitor_team, game_ids))

In [183]:
print(avg_fg310_team_home_rs_03)
print(len(avg_fg310_team_home_rs_03) == len(df_rs_03))

[0.35, 0.1, 0.35, 0.313, 0.313, 0.16, 0.1669999999999999, 0.3329999999999999, 0.1669999999999999, 0.231, 0.4, 0.556, 0.238, 0.222, 0.31, 0.28300000000000003, 0.5710000000000001, 0.35, 0.2499999999999999, 0.25933333333333336, 0.375, 0.2485, 0.231, 0.3655, 0.3905, 0.3835, 0.258, 0.32766666666666666, 0.39066666666666666, 0.3426666666666667, 0.25566666666666665, 0.29633333333333334, 0.27133333333333337, 0.35200000000000004, 0.19566666666666666, 0.36933333333333335, 0.39033333333333337, 0.36649999999999994, 0.3115, 0.28933333333333333, 0.30825, 0.37625, 0.25933333333333336, 0.3525, 0.322, 0.37966666666666665, 0.3353333333333333, 0.259, 0.31475, 0.34075, 0.2508, 0.2722, 0.30175, 0.3266, 0.18974999999999997, 0.39075, 0.29974999999999996, 0.32799999999999996, 0.30974999999999997, 0.27525, 0.3765, 0.3946, 0.32974999999999993, 0.28683333333333333, 0.3128, 0.244, 0.24059999999999998, 0.3115, 0.32116666666666666, 0.36749999999999994, 0.41259999999999997, 0.3775, 0.35800000000000004, 0.354, 0.3126,

In [184]:
#4)
df_rs_03['AVG_FG3PCT_LAST10_HOME_TEAM'] = pd.Series(avg_fg310_team_home_rs_03)
df_rs_03['AVG_FG3PCT_LAST10_VISITOR_TEAM'] = pd.Series(avg_fg310_team_visitor_rs_03)
df_rs_03

/tmp/ipykernel_4387/222365465.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_LAST10_HOME_TEAM'] = pd.Series(avg_fg310_team_home_rs_03)
/tmp/ipykernel_4387/222365465.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rs_03['AVG_FG3PCT_LAST10_VISITOR_TEAM'] = pd.Series(avg_fg310_team_visitor_rs_03)


,GAME_DATE,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_ABBREVIATION,VISITOR_TEAM_NICKNAME,SEASON,PTS_home,...,AVG_FGPERCENT_LAST5_HOME_TEAM,AVG_FGPERCENT_LAST5_VISITOR_TEAM,AVG_FGPERCENT_LAST10_HOME_TEAM,AVG_FGPERCENT_LAST10_VISITOR_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_HOME_TEAM,AVG_FG3PCT_UNTIL_THIS_GAME_VISITOR_TEAM,AVG_FG3PCT_LAST5_HOME_TEAM,AVG_FG3PCT_LAST5_VISITOR_TEAM,AVG_FG3PCT_LAST10_HOME_TEAM,AVG_FG3PCT_LAST10_VISITOR_TEAM
0,2003-10-28,20300001,1610612755,1610612748,PHI,76ers,MIA,Heat,2003,89.0,...,0.4400,0.4080,0.4400,0.4080,0.350000,0.250000,0.3500,0.2500,0.3500,0.2500
1,2003-10-28,20300002,1610612759,1610612756,SAS,Spurs,PHX,Suns,2003,83.0,...,0.4250,0.3610,0.4250,0.3610,0.100000,0.500000,0.1000,0.5000,0.1000,0.5000
2,2003-10-28,20300003,1610612747,1610612742,LAL,Lakers,DAL,Mavericks,2003,109.0,...,0.5060,0.3760,0.5060,0.3760,0.350000,0.280000,0.3500,0.2800,0.3500,0.2800
3,2003-10-29,20300004,1610612738,1610612748,BOS,Celtics,MIA,Heat,2003,98.0,...,0.5070,0.3870,0.5070,0.3870,0.313000,0.333500,0.3130,0.3335,0.3130,0.3335
4,2003-10-29,20300005,1610612752,1610612753,NYK,Knicks,ORL,Magic,2003,83.0,...,0.4000,0.3680,0.4000,0.3680,0.313000,0.250000,0.3130,0.2500,0.3130,0.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2004-04-14,20301185,1610612762,1610612756,UTA,Jazz,PHX,Suns,2003,84.0,...,0.4114,0.4424,0.4363,0.4537,0.318878,0.340049,0.3044,0.4792,0.4272,0.3745
1185,2004-04-14,20301186,1610612745,1610612742,HOU,Rockets,DAL,Mavericks,2003,89.0,...,0.4638,0.4686,0.4612,0.5147,0.365305,0.347183,0.4136,0.3966,0.3748,0.4257
1186,2004-04-14,20301187,1610612757,1610612747,POR,Trail Blazers,LAL,Lakers,2003,104.0,...,0.4330,0.4402,0.4659,0.4456,0.343756,0.321244,0.3812,0.2542,0.3410,0.3458
1187,2004-04-14,20301188,1610612746,1610612760,LAC,Clippers,OKC,Thunder,2003,87.0,...,0.4160,0.4930,0.4170,0.4799,0.317146,0.373902,0.1910,0.3962,0.2757,0.4005
